In [1]:
# importing package
from __future__ import division
import os
import re
import sys
import nltk
import gensim
import pprint
import shutil
import PyPDF2
import textract
import unicodedata
# importing package functions
from nltk import *
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#reload(sys)
#sys.setdefaultencoding('utf8')
# Define base files to compare

In [2]:
path = '/home/sistemas/Compras/'
base = path+'Pedidos Mayoristas/Pedidos Mayoristas 2017/Makicop/'
recover = path+'Recuperados/pdf/renombrar'
compareFiles = []
raw_documents = []

In [3]:
for i in os.listdir(base):
    ruta = base+i
    if not os.path.isdir(ruta) and ".pdf" in i:
        compareFiles.append(ruta)
    elif os.path.isdir(ruta):
        for j in os.listdir(ruta):
            subruta = ruta+"/"+j
            if not os.path.isdir(subruta) and ".pdf" in j:
                compareFiles.append(subruta)
            elif os.path.isdir(subruta):
                for k in os.listdir(subruta):
                    subsubruta = subruta+"/"+k
                    if not os.path.isdir(subsubruta) and ".pdf" in k:
                        compareFiles.append(subsubruta)

In [4]:
print compareFiles[17]

/home/sistemas/Compras/Pedidos Mayoristas/Pedidos Mayoristas 2017/Makicop/Pendientes Makicop 2016-2017/Estado de Cuenta y Respaldos Makicop.pdf


In [5]:
grateCounter = 0

for i in compareFiles:
    grateCounter += 1
    print grateCounter," \r",
    count = 0
    text = ""
    pdfFileObj = open(i,'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    num_pages = pdfReader.numPages
    while count < num_pages:
        pageObj = pdfReader.getPage(count)
        count +=1
        text += pageObj.extractText()
    if text != "":
        text = text
        text=unicodedata.normalize('NFKD',text).encode('ascii','ignore')
        raw_documents.append(text)
    else:
        #print type(i)
        text = textract.process(i, method='tesseract', language='eng', encoding='ascii')
        #print text
        raw_documents.append(text)

In [6]:
gen_docs = [[w.lower() for w in word_tokenize(text)] 
            for text in raw_documents]

In [7]:
print gen_docs

[['makicop', ',', 'sa', 'de', 'cv', 'i', 'morena', 'no', '.', '108', 'dei', 'vaiie', 'centro', '_', 'benlto', 'juarez', 'cludad', 'de', 'mexico', ',', 'gp', '03100', 'mak911213km3', 'www.makicop', 'com', 'mx', 'tei', '.', '5543-1510', 'factura', 'm', '9129', 'ientews366', 'w', "''", 'erecv', 'receetor', ';', '7ed8050817273k0', 'ww', "''", 'i=echa', 'yam', ';', '7', "24/02/2017t09'03", '2177', 'i', 'eao', 'digital', 'solutions', '.', 's', 'a', '.', 'de', 'cv', '.', 'fecha', 'y', 'no', '24/02/2', '7t09', 'as', '36', '7*', 'we', "5'.", '``', '54ma96593033-754f-', '<', 'domicilio', 'fiscal', '>', 'iqqd7-17sb4fd67fac', 'sebastian', 'del', 'piombo', 'no', '.', '82', 'int_', '2', 'no', 'cenmcado', 'sat', 'i0000|000000401041203', 'nonoalco', 'no', 'cemncado', 'oucmooooommmsasm', 'benito', 'juarez', 'melodo', 'de', 'page', ':', '93', 'na', 'ciudad', 'de', 'mxico', 'gp', '03700', 'cuenla', ':', 'lugar', 'de', 'expedicin', 'ciudad', 'de', 'mxlco', 'rgimen', 'generai', 'de', 'ley', 'de', 'personas

In [8]:
dictionary = gensim.corpora.Dictionary(gen_docs)

In [9]:
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]

In [10]:
tf_idf = gensim.models.TfidfModel(corpus)

In [11]:
sims = gensim.similarities.Similarity('/home/sistemas/Documentos/Scripts/',tf_idf[corpus],
                                      num_features=len(dictionary))

In [12]:
grateCounter = 0
resultados = []

for i in os.listdir(recover):
        if ".pdf" in i:
            try:
                grateCounter += 1
                incidencia = 0
                print "\n",grateCounter,"::",recover+"/"+i,"\n"
                pdfFileObj = open(recover+"/"+i,'rb')
                pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
                num_pages = pdfReader.numPages
                count = 0
                text = ""
                while count < num_pages:
                        pageObj = pdfReader.getPage(count)
                        count +=1
                        text += pageObj.extractText()
                if text != "":
                    text = text
                    text=unicodedata.normalize('NFKD',text).encode('ascii','ignore')
                    raw_documents.append(text)
                else:
                    #print "using tesseract"
                    text = textract.process(recover+"/"+i, method='tesseract', language='eng', encoding='ascii')
                    raw_documents.append(text)                    
                query_doc = [w.lower() for w in word_tokenize(text)]
                query_doc_bow = dictionary.doc2bow(query_doc)
                query_doc_tf_idf = tf_idf[query_doc_bow]
                sims[query_doc_tf_idf]
                #if sims[query_doc_tf_idf][0] >= 0.8:
                #        print "Probablemente ",i," sea una factura"
                #else:
                #E        print "Probablemente ",i," no lo sea"
                #        #shutil.move(path+i,path+"/nofac/"+i)
                for j in sims[query_doc_tf_idf]:
                    if j >= .5 and incidencia == 0:
                        print sims[query_doc_tf_idf].tolist().index(j),":",j
                        resultados.append(str(sims[query_doc_tf_idf].tolist().index(j))+":"+str(j))
                        shutil.move(recover+"/"+i,recover+"/maki/"+i)
                        incidencia+=1
            except:
                pass


1 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f510029832.pdf 


2 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f370025528.pdf 


3 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f23188480.pdf 


4 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f404923888.pdf 


5 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f693891560.pdf 


6 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f369740400.pdf 


7 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f621514440.pdf 


8 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f294269416.pdf 


9 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f35801144.pdf 


10 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f606347488.pdf 


11 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f643222528.pdf 


12 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f568048968.pdf 


13 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f663063888.pdf 


14 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f53750


116 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f366131040.pdf 


117 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f671510944.pdf 


118 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f438238208.pdf 


119 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f535309776.pdf 


120 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f690849704.pdf 


121 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f576806944.pdf 


122 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f371013200.pdf 


123 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f476390472.pdf 


124 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f691007720.pdf 


125 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f657182640.pdf 


126 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f639172184.pdf 


127 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f476568168.pdf 


128 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f508554000.pdf 


129 :: /home/sistemas/Compras/Recuper


228 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f438275456.pdf 


229 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f656989024.pdf 


230 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f696082976.pdf 


231 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f442822552.pdf 


232 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f471701224.pdf 


233 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f597074336.pdf 


234 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f275268704.pdf 


235 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f362178264.pdf 


236 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f596677824.pdf 


237 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f594152384.pdf 


238 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f445580584.pdf 


239 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f592535296.pdf 


240 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f669154656.pdf 


241 :: /home/sistemas/Compras/Recuper


274 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f690618784.pdf 


275 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f370003960.pdf 


276 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f670373752.pdf 


277 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f593180608.pdf 


278 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f567720896.pdf 


279 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f286404016.pdf 


280 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f671587256.pdf 


281 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f679920120.pdf 


282 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f552136192.pdf 


283 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f408679280.pdf 


284 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f596749928.pdf 


285 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f370001128.pdf 


286 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f450528040.pdf 


287 :: /home/sistemas/Compras/Recuper


386 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f695691128.pdf 


387 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f434251104.pdf 


388 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f646271320.pdf 


389 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f577542768.pdf 


390 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f408458152.pdf 


391 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f408856528.pdf 


392 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f512228872.pdf 


393 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f510294256.pdf 


394 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f657487232.pdf 


395 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f596620392.pdf 


396 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f646380576.pdf 


397 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f568054600.pdf 


398 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f563009360.pdf 


399 :: /home/sistemas/Compras/Recuper

...


6850 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f534903392.pdf 


6851 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f396819352.pdf 


6852 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f277606720.pdf 


6853 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f505642992.pdf 


6854 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f701368720.pdf 


6855 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f597071104.pdf 


6856 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f286135464.pdf 


6857 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f607518608.pdf 


6858 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f362454192.pdf 


6859 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f382850920.pdf 


6860 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f419793192.pdf 


6861 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f535710472.pdf 


6862 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f656642176.pdf 


6863 :: /home/sistemas/C


6960 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f463125768.pdf 


6961 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f644586024.pdf 


6962 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f617683840.pdf 


6963 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f645712576.pdf 


6964 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f621989912.pdf 


6965 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f549095280.pdf 


6966 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f563923824.pdf 


6967 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f632673152.pdf 


6968 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f286506472.pdf 


6969 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f277489960.pdf 


6970 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f607088112.pdf 


6971 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f422278656.pdf 


6972 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f353731752.pdf 


6973 :: /home/sistemas/C


7073 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f476301656.pdf 


7074 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f670389968.pdf 


7075 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f409511784.pdf 


7076 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f489303696.pdf 


7077 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f568163712.pdf 


7078 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f445215424.pdf 


7079 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f617424000.pdf 


7080 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f478753192.pdf 


7081 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f293998864.pdf 


7082 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f701249536.pdf 


7083 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f696611824.pdf 


7084 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f534612632.pdf 


7085 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f365758800.pdf 


7086 :: /home/sistemas/C


7184 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f690665808.pdf 


7185 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f276699544.pdf 


7186 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f286571312.pdf 


7187 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f539102680.pdf 


7188 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f472801920.pdf 


7189 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f667652264.pdf 


7190 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f577252128.pdf 


7191 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f491145472_ReportedeRefacciones_(14).pdf 


7192 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f596640016.pdf 


7193 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f442809984.pdf 


7194 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f476368640.pdf 


7195 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f668275008.pdf 


7196 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f329038088.pdf 



7293 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f476250720.pdf 


7294 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f671819080.pdf 


7295 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f644831608.pdf 


7296 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f445581368.pdf 


7297 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f539099136.pdf 


7298 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f643233712.pdf 


7299 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f567957096.pdf 


7300 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f592722200.pdf 


7301 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f632512896.pdf 


7302 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f596764824.pdf 


7303 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f278124016.pdf 


7304 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f593423168.pdf 


7305 :: /home/sistemas/Compras/Recuperados/pdf/renombrar/f509011568.pdf 


7306 :: /home/sistemas/C